In [1]:
import nltk
import codecs
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk.data  
from gensim.models import word2vec

C:\Users\Kailash Nathan\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
dataset = pd.read_csv('ReviewsClean.csv')

In [5]:
dataset.drop('Unnamed: 0', axis=1, inplace=True)

In [40]:
data=dataset[:100000]

In [41]:
stars=data['stars']

In [42]:
reviews=data['text']

In [43]:
stars.shape,reviews.shape,dataset.shape

((100000L,), (100000L,), (4037798, 2))

In [44]:
X_tr, X_te, Y_tr, Y_te = train_test_split(reviews,stars, test_size=0.1,shuffle=True, random_state=1)

In [45]:
X_tr.shape

(90000L,)

In [46]:
dataset.shape,X_tr.shape,Y_tr.shape

((4037798, 2), (90000L,), (90000L,))

In [47]:
stars.shape,X_te.shape,Y_te.shape

((100000L,), (10000L,), (10000L,))

In [48]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [49]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    #raw_sentences1 = tokenizer.tokenize(review)
    #print "old_len:",len(raw_sentences1)
    #raw_sentences = review.split("\n")
    #print "new_len:",len(raw_sentences)
    # 2. Break sentences into separate 
    #raw_sentences = raw_sentences.split("\n")
    #
    # 3. Loop over each sentence
    sentences = []
    counter = 0
    for raw_sentence in review:
        counter += 1
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
        if (counter%10000 == 0):
            print ("Done with %d Reviews"%counter)
        
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences






In [50]:
review_to_sentences(X_tr.values[2:4],)

TypeError: review_to_sentences() takes at least 2 arguments (1 given)

In [51]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords == 0:
        return featureVec 
    else:
        return np.divide(featureVec,nwords)


In [52]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%10000 == 0:
           print ("Review %d of %d" % (counter, len(reviews)))
           
           
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs





In [53]:
# Initialize paths for training data set


#train_labels = "traindata.labels"

#Open Training files, get content and close files
#tr_labels = ftr_l.read()
#ftr_d.close()
#ftr_l.close()

print ("All set with the files now")

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

sentences = []  # Initialize an empty list of sentences
print ("Parsing sentences from training set")

#print X_tr.values[9]
sentences = review_to_sentences(X_tr.values,tokenizer)

#

All set with the files now
Parsing sentences from training set
Done with 10000 Reviews
Done with 20000 Reviews
Done with 30000 Reviews
Done with 40000 Reviews
Done with 50000 Reviews
Done with 60000 Reviews
Done with 70000 Reviews
Done with 80000 Reviews
Done with 90000 Reviews


In [54]:
print len(sentences),len(X_tr.values)

90000 90000


In [55]:
#print "len of sentences:",len(sentences)

#print sentences[0]


##### Training the word2vec model

# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)

print( "Training model...")

model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
#model_name = "300features_40minwords_10context"
#model.save(model_name)

# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
print ("Creating average feature vecs for training set")
#clean_train_reviews = review_to_sentences( tr_data, tokenizer, \
#        remove_stopwords=True )

trainDataVecs = getAvgFeatureVecs( sentences, model, num_features )



2017-12-07 20:04:02,686 : INFO : collecting all words and their counts
2017-12-07 20:04:02,687 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2017-12-07 20:04:03,167 : INFO : PROGRESS: at sentence #10000, processed 1140958 words, keeping 25959 word types
2017-12-07 20:04:03,641 : INFO : PROGRESS: at sentence #20000, processed 2291219 words, keeping 35263 word types
2017-12-07 20:04:04,059 : INFO : PROGRESS: at sentence #30000, processed 3420525 words, keeping 42001 word types
2017-12-07 20:04:04,404 : INFO : PROGRESS: at sentence #40000, processed 4552917 words, keeping 47458 word types
2017-12-07 20:04:04,852 : INFO : PROGRESS: at sentence #50000, processed 5686041 words, keeping 51978 word types
2017-12-07 20:04:05,213 : INFO : PROGRESS: at sentence #60000, processed 6823209 words, keeping 56004 word types
2017-12-07 20:04:05,690 : INFO : PROGRESS: at sentence #70000, processed 7956870 words, keeping 59760 word types
2017-12-07 20:04:06,210 : INFO : PROGRESS: at sentence #80000, processed 9098924 words, keeping 63298 word types
2017-12-07 20:04:06,486 : INFO : collected 66580 word types from a corpus of 10233569 raw words 

2017-12-07 20:05:11,421 : INFO : PROGRESS: at 89.08% examples, 509590 words/s, in_qsize 7, out_qsize 0
2017-12-07 20:05:12,437 : INFO : PROGRESS: at 90.65% examples, 510389 words/s, in_qsize 8, out_qsize 0
2017-12-07 20:05:13,438 : INFO : PROGRESS: at 92.19% examples, 511079 words/s, in_qsize 8, out_qsize 0
2017-12-07 20:05:14,443 : INFO : PROGRESS: at 93.69% examples, 511646 words/s, in_qsize 8, out_qsize 0
2017-12-07 20:05:15,450 : INFO : PROGRESS: at 95.58% examples, 514122 words/s, in_qsize 8, out_qsize 0
2017-12-07 20:05:16,457 : INFO : PROGRESS: at 97.68% examples, 517716 words/s, in_qsize 7, out_qsize 0
2017-12-07 20:05:17,467 : INFO : PROGRESS: at 99.76% examples, 521163 words/s, in_qsize 7, out_qsize 0
2017-12-07 20:05:17,548 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-12-07 20:05:17,549 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-07 20:05:17,569 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-1

Creating average feature vecs for training set
Review 0 of 90000
Review 10000 of 90000
Review 20000 of 90000
Review 30000 of 90000
Review 40000 of 90000
Review 50000 of 90000
Review 60000 of 90000
Review 70000 of 90000
Review 80000 of 90000


In [56]:
#fte_d = codecs.open('testdatawv','r',encoding='utf-8')
#te_data   = fte_d.read()
test_data=X_te.values
print ("Creating average feature vecs for test set")
clean_test_reviews = review_to_sentences( X_te.values, tokenizer, \
        remove_stopwords=True )

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )



Creating average feature vecs for test set
Done with 10000 Reviews
Review 0 of 10000


In [57]:
trainDataVecs

array([[-0.00025731, -0.01680092,  0.00895932, ..., -0.0142184 ,
        -0.01067297, -0.00277135],
       [ 0.00044126, -0.00732648,  0.00196551, ..., -0.00181366,
        -0.01030169,  0.02095335],
       [ 0.01188463, -0.0180675 , -0.00353899, ...,  0.0101138 ,
         0.01402176, -0.01768884],
       ..., 
       [ 0.00611622, -0.00618228, -0.008958  , ..., -0.01163218,
        -0.01446202,  0.01348171],
       [ 0.01544579, -0.00058106, -0.00848628, ...,  0.00437743,
        -0.00975345, -0.00526944],
       [-0.00898447, -0.0242632 ,  0.00608002, ...,  0.00534351,
        -0.00244264,  0.00706471]], dtype=float32)

In [58]:


forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
listlabels = Y_tr.to_string().strip().split("\n")
print len(listlabels)
print len(testDataVecs)
forest = forest.fit( trainDataVecs, Y_tr )
print "train done"
# Test & extract results 
result = forest.predict( testDataVecs[:1000] )
print "test done"
#Make a list of the ids
#fTest_ids = open('testlabel','r')#,encoding='utf=8')
#all_ids = fTest_ids.read()
#fTest_ids.close()
idsList = Y_te.to_string().strip().split("\n")
print len(idsList)
# Write the test results 
#output = []
#output = pd.DataFrame( data={"id":idsList, "prediction:":result, "gold-label:":labelslist} )
#output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )
acc = 0.0
for a,b in zip(result,Y_te):
    if a == b:
        acc += 1.0

print "Test set accuracy=", acc / len(result)



Fitting a random forest to labeled training data...
90000
10000
train done
test done
10000
Test set accuracy= 0.63


In [59]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
forest = RandomForestClassifier(n_estimators = 100) 

In [61]:
listlabels = Y_tr.to_string().strip().split("\n")
idsList = Y_te.to_string().strip().split("\n")

In [62]:
forest = forest.fit( trainDataVecs,Y_tr )

In [63]:
pre=forest.predict( testDataVecs)

In [64]:
print pre

[4 5 1 ..., 5 5 5]


In [65]:
from sklearn.metrics import confusion_matrix, classification_report

In [66]:
print (classification_report(Y_te, pre))

             precision    recall  f1-score   support

          1       0.70      0.69      0.69      1503
          2       0.53      0.07      0.13       992
          4       0.50      0.29      0.36      2686
          5       0.63      0.89      0.74      4819

avg / total       0.60      0.62      0.57     10000

